In [2]:
import os
import glob
import numpy as np
import pandas as pd


# =========================
# 0) 설정 (여기만 수정)
# =========================
DATA_DIR = r"C:\Users\Jeon\sesac-miniProject\완료"
START_DATE = pd.to_datetime("2025-01-14").date()
END_DATE   = pd.to_datetime("2026-01-14").date()

OUT_DIR = os.path.join(DATA_DIR, "daily_outputs")
os.makedirs(OUT_DIR, exist_ok=True)

# 파일명에 포함된 키워드로 자동 매칭 (현대차 제외)
TARGETS = {
    "삼성": ["삼성", "samsung"],
    "하이닉스": ["하이닉스", "sk하이닉스", "hynix"],
    "현대차": ["현대차", "현대 자동차", "hyundai"]
}

# ✅ 과열지수(OI) 가중치
W_VIEWS    = 0.25
W_POSTS    = 0.25
W_COMMENTS = 0.30
W_LIKES    = 0.20




# =========================
# 1) 유틸
# =========================
def to_int_series(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.replace(",", "", regex=False)
    s = s.str.extract(r"(\d+)")[0].fillna("0")
    return s.astype(int)


def find_file_for_target(csv_files, keywords):
    lower_map = [(f, os.path.basename(f).lower()) for f in csv_files]
    for kw in keywords:
        kw_l = kw.lower()
        for f, base in lower_map:
            if kw_l in base:
                return f
    return None


def daily_aggregate(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path, encoding="utf-8-sig")

    # ✅ 네 CSV 컬럼명(한글) -> 내부 표준 컬럼명(영문)으로 매핑
    rename_map = {
        "날짜": "date",
        "제목": "title",
        "조회": "views",
        "추천": "likes",
        "댓글수": "comments",
    }
    df = df.rename(columns=rename_map)

    # ✅ 날짜 포맷이 이미 'YYYY-MM-DD'로 고정이라면, 추가 전처리 없이 바로 파싱
    dt = pd.to_datetime(df["date"], errors="coerce")  # 문자열 -> datetime [web:231]

    df = df.copy()
    df["date_dt"] = dt.dt.date
    df = df.dropna(subset=["date_dt"])
    df = df[(df["date_dt"] >= START_DATE) & (df["date_dt"] <= END_DATE)]

    if "title" not in df.columns:
        df["title"] = ""
    for col in ["views", "likes", "comments"]:
        if col not in df.columns:
            df[col] = 0

    df["views"] = to_int_series(df["views"])
    df["likes"] = to_int_series(df["likes"])
    df["comments"] = to_int_series(df["comments"])

    daily = (
        df.groupby("date_dt", as_index=False)
          .agg(
              posts=("title", "size"),
              views=("views", "sum"),
              comments=("comments", "sum"),
              likes=("likes", "sum"),
          )
    )

    # ✅ 기간 전체 날짜를 채움(글 없는 날 0)
    full = pd.DataFrame({"date_dt": pd.date_range(START_DATE, END_DATE, freq="D").date})
    daily = full.merge(daily, on="date_dt", how="left").fillna(0)

    for c in ["posts", "views", "comments", "likes"]:
        daily[c] = daily[c].astype(int)

    daily = daily.sort_values("date_dt")
    daily["date"] = pd.to_datetime(daily["date_dt"]).dt.strftime("%Y-%m-%d")

    daily = daily[["date", "posts", "views", "comments", "likes"]].rename(columns={
        "date": "날짜",
        "posts": "게시글수",
        "views": "조회수",
        "comments": "댓글수",
        "likes": "좋아요수",
    })
    return daily


def summarize_basic(daily: pd.DataFrame):
    max_posts = int(daily["게시글수"].max())
    top_days = daily[daily["게시글수"] == max_posts][["날짜", "게시글수", "조회수", "댓글수", "좋아요수"]]

    avg_daily_views = float(daily["조회수"].mean())
    std_daily_views = float(daily["조회수"].std(ddof=1))

    return {
        "기간일수": int(len(daily)),
        "글있는날": int((daily["게시글수"] > 0).sum()),
        "총게시글": int(daily["게시글수"].sum()),
        "일평균조회수": avg_daily_views,
        "일조회수_표준편차": std_daily_views,
        "최다게시글수(하루)": max_posts,
        "최다게시글_날짜들": ", ".join(top_days["날짜"].tolist()),
    }, top_days


def zscore(series: pd.Series) -> pd.Series:
    mu = series.mean()
    sd = series.std(ddof=1)
    if sd == 0 or np.isnan(sd):
        return pd.Series(np.zeros(len(series)), index=series.index)
    return (series - mu) / sd


def add_overheat_index(daily: pd.DataFrame) -> pd.DataFrame:
    d = daily.copy()

    d["조회수_z"] = zscore(d["조회수"])
    d["게시글수_z"] = zscore(d["게시글수"])
    d["댓글수_z"] = zscore(d["댓글수"])
    d["좋아요수_z"] = zscore(d["좋아요수"])

    d["과열지수_OI"] = (
        W_VIEWS    * d["조회수_z"] +
        W_POSTS    * d["게시글수_z"] +
        W_COMMENTS * d["댓글수_z"] +
        W_LIKES    * d["좋아요수_z"]
    )
    return d


def summarize_oi(daily_with_oi: pd.DataFrame):
    max_oi = daily_with_oi["과열지수_OI"].max()
    top_oi_days = daily_with_oi[daily_with_oi["과열지수_OI"] == max_oi][
        ["날짜", "과열지수_OI", "게시글수", "조회수", "댓글수", "좋아요수"]
    ].copy()

    return {
        "과열지수_OI_평균": float(daily_with_oi["과열지수_OI"].mean()),
        "과열지수_OI_표준편차": float(daily_with_oi["과열지수_OI"].std(ddof=1)),
        "과열지수_OI_최댓값": float(max_oi),
        "과열지수_OI_최대날짜들": ", ".join(top_oi_days["날짜"].tolist()),
    }, top_oi_days


# =========================
# 2) 실행: 파일 자동 찾기
# =========================
csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
if not csv_files:
    raise FileNotFoundError(f"'{DATA_DIR}' 폴더에 CSV가 없습니다. DATA_DIR을 확인하세요.")

picked = {}
missing = []
for name, kws in TARGETS.items():
    f = find_file_for_target(csv_files, kws)
    if f:
        picked[name] = f
    else:
        missing.append(name)

print("✅ 자동 인식된 파일")
for k, v in picked.items():
    print(f" - {k}: {os.path.basename(v)}")

if missing:
    print("\n⚠️ 못 찾은 대상:", ", ".join(missing))
    print("   파일명이 너무 다르면 TARGETS 키워드를 추가하거나 파일명을 바꿔주세요.\n")


# =========================
# 3) 종목별: 일별 집계 + OI 계산 + 저장 (CSV만)
# =========================
daily_tables = {}
summary_rows = []
top_posts_all = []
top_oi_all = []

for stock, path in picked.items():
    daily = daily_aggregate(path)
    daily_oi = add_overheat_index(daily)
    daily_tables[stock] = daily_oi

    # 종목별 CSV 저장 (OI 포함)
    out_csv = os.path.join(OUT_DIR, f"{stock}_일별집계_OI포함_{START_DATE}_{END_DATE}.csv")
    daily_oi.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"[저장] {stock} 일별집계(OI포함) CSV -> {out_csv}")

    # 요약용 데이터 만들기
    basic_summ, top_days = summarize_basic(daily)
    oi_summ, top_oi_days = summarize_oi(daily_oi)

    summary_rows.append({
        "종목": stock,
        "원본파일": os.path.basename(path),
        **basic_summ,
        **oi_summ
    })

    tmp1 = top_days.copy()
    tmp1.insert(0, "종목", stock)
    top_posts_all.append(tmp1)

    tmp2 = top_oi_days.copy()
    tmp2.insert(0, "종목", stock)
    top_oi_all.append(tmp2)

summary_df = pd.DataFrame(summary_rows)

# 요약 CSV 저장 (파일명은 그대로 두되, 실제 내용은 2종목만 들어감)
summary_csv = os.path.join(OUT_DIR, f"2종목_요약_OI포함_{START_DATE}_{END_DATE}.csv")
summary_df.to_csv(summary_csv, index=False, encoding="utf-8-sig")
print(f"[저장] 요약 CSV -> {summary_csv}")

# (선택) 최다 게시글 날짜들/최대 OI 날짜들도 CSV로 저장
# if top_posts_all:
#     top_posts_csv = os.path.join(OUT_DIR, f"최다게시글날짜_{START_DATE}_{END_DATE}.csv")
#     pd.concat(top_posts_all, ignore_index=True).to_csv(top_posts_csv, index=False, encoding="utf-8-sig")
#     print(f"[저장] 최다 게시글 날짜 CSV -> {top_posts_csv}")

# if top_oi_all:
#     top_oi_csv = os.path.join(OUT_DIR, f"최대OI날짜_{START_DATE}_{END_DATE}.csv")
#     pd.concat(top_oi_all, ignore_index=True).to_csv(top_oi_csv, index=False, encoding="utf-8-sig")
#     print(f"[저장] 최대 OI 날짜 CSV -> {top_oi_csv}")


print("\n==== 화면 요약(핵심만) ====")
print(summary_df[
    ["종목",
     "총게시글",
     "최다게시글수(하루)",
     "최다게시글_날짜들",
     "일평균조회수",
     "일조회수_표준편차",
     "과열지수_OI_최댓값",
     "과열지수_OI_최대날짜들",
     "과열지수_OI_평균",
     "과열지수_OI_표준편차"
     ]
])


✅ 자동 인식된 파일
 - 현대차: fm_hyundai_normal.csv

⚠️ 못 찾은 대상: 삼성, 하이닉스
   파일명이 너무 다르면 TARGETS 키워드를 추가하거나 파일명을 바꿔주세요.

[저장] 현대차 일별집계(OI포함) CSV -> C:\Users\Jeon\sesac-miniProject\완료\daily_outputs\현대차_일별집계_OI포함_2025-01-14_2026-01-14.csv
[저장] 요약 CSV -> C:\Users\Jeon\sesac-miniProject\완료\daily_outputs\2종목_요약_OI포함_2025-01-14_2026-01-14.csv

==== 화면 요약(핵심만) ====
    종목   총게시글  최다게시글수(하루)   최다게시글_날짜들        일평균조회수     일조회수_표준편차  \
0  현대차  10793         402  2026-01-07  29985.505464  55191.225279   

   과열지수_OI_최댓값 과열지수_OI_최대날짜들    과열지수_OI_평균  과열지수_OI_표준편차  
0     6.755664    2025-10-30  3.882747e-17      0.989209  


In [2]:
print("on")

on
